# btlflt.ipynb

In [16]:
import os

In [17]:
# Note: ```.gitignore``` includes VIDEOPATH and FRAMESDIR

VIDEOPATH = 'video.mp4'
FRAMESDIR = 'frames'

In [18]:
# Step 1: Download the video (best quality, no audio)

if os.path.exists(VIDEOPATH):
    pass
else:
    command = f'yt-dlp -o {VIDEOPATH} -f bestvideo https://www.youtube.com/watch?v=IUQSZHr9h9Q'
    os.system(command)

In [19]:
# Step 2: Extract frames from the video

if os.path.exists(FRAMESDIR):
    pass
else:
    os.mkdir(FRAMESDIR)
    command = f'ffmpeg -r 1 -i {VIDEOPATH} -r 1 {FRAMESDIR}/%04d.png'
    os.system(command)